In [1]:
import numpy as np
import pandas as pd
from sigmoid import sigmoid
from cost_function import *
import scipy.optimize as opt
import os
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from predict import *
from sklearn.metrics import *

In [2]:
PATH_INPUT = os.getcwd() + '/inputs/'

### Implementação das funções de custo

In [263]:
def cal_cost(theta, X, y):
    """
        :input: theta -
        :input: X -
        :input: y -
        
        :return: cost
    """
    m = len(y)
    
    predictions = X.dot(theta)
    cost = (1/2*m) * np.sum(np.square(predictions-y))
    
    return cost

In [264]:
def gradient_descent(theta,X,y,learning_rate=0.01,iterations=100):
    """
        :input: X - 
        :input: X - 
        :input: X - 
        :input: X - 
        :input: X - 
        
        :return: theta - 
        :return: cost_history - 
        :return: theta_history -         
    """
    m = len(y)
    cost_history = np.zeros(iterations)
    theta_history = np.zeros((iterations,3))
    for it in range(iterations):
        prediction = np.dot(X,theta)
        theta = theta -(1/m)*learning_rate*( X.T.dot((prediction - y)))
        theta_history[it,:] =theta.T
        cost_history[it]  = cal_cost(theta,X,y)

    return theta, cost_history, theta_history

In [286]:
def minibatch_gradient_descent(X,y,theta,learning_rate=0.01,iterations=10,batch_size =20):
    '''
    X    = Matrix of X without added bias units
    y    = Vector of Y
    theta=Vector of thetas np.random.randn(j,1)
    learning_rate 
    iterations = no of iterations

    Returns the final theta vector and array of cost history over no of iterations
    '''
    m = len(y)
    cost_history = np.zeros(iterations)
    n_batches = int(m/batch_size)

    for it in range(iterations):
        cost =0.0
        indices = np.random.permutation(m)
        X = X[indices]
        y = y[indices]
        for i in range(0,m,batch_size):
            X_i = X[i:i+batch_size]
            y_i = y[i:i+batch_size]

            X_i = np.c_[np.ones(len(X_i)),X_i]

            prediction = np.dot(X_i,theta)

            theta = theta -(1/m)*learning_rate*( X_i.T.dot((prediction - y_i)))
            cost += cal_cost(theta,X_i,y_i)
        cost_history[it]  = cost

    return theta, cost_history

In [46]:
def predict(theta, X):

    predict = sigmoid(np.dot(X, theta))
    predict = 1 * (predict >= 0.5)

    return predict

### Load do dataset de treino e separacao dos dados

Load do cvs com o treino feito anteriormente

In [298]:
df_treino = pd.read_csv(PATH_INPUT+'training-data.csv')
df_treino

,character_name,episode_appearance_count,kill_count,dead
0,Addam Marbrand,1,0,0
1,Aeron Greyjoy,2,0,0
2,Akho,3,0,1
3,Alliser Thorne,19,1,1
4,Alton Lannister,3,0,1
5,Alys Karstark,4,0,0
6,Amory Lorch,3,1,1
7,Anguy,6,0,0
8,Areo Hotah,6,0,1
9,Armeca,4,0,0


Normaliza o X

In [299]:
std_scale = preprocessing.MinMaxScaler()

df_treino['episode_appearance_count'] = std_scale.fit_transform((df_treino['episode_appearance_count'].values).reshape((-1,1)))
df_treino['kill_count'] = std_scale.fit_transform((df_treino['kill_count'].values).reshape((-1,1)))

df_treino

/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


,character_name,episode_appearance_count,kill_count,dead
0,Addam Marbrand,0.000000,0.000000,0
1,Aeron Greyjoy,0.016129,0.000000,0
2,Akho,0.032258,0.000000,1
3,Alliser Thorne,0.290323,0.071429,1
4,Alton Lannister,0.032258,0.000000,1
5,Alys Karstark,0.048387,0.000000,0
6,Amory Lorch,0.032258,0.071429,1
7,Anguy,0.080645,0.000000,0
8,Areo Hotah,0.080645,0.000000,1
9,Armeca,0.048387,0.000000,0


Carrega o X e y

In [300]:
X = df_treino.iloc[:,1:3]
y = df_treino.iloc[:,3:]

Split do dataframe em treino e teste

In [301]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, shuffle=True)

### Procurando o theta ...

In [302]:
x_theta = np.c_[np.ones((X_train.shape[0],1)),X_train.values]
y_theta = y_train.values

In [303]:
initial_theta = np.random.rand(3,1)
initial_theta

array([[0.01728661],
       [0.93632198],
       [0.81966902]])

In [304]:
theta_grad, cost_history, theta_history = gradient_descent(initial_theta,x_theta,y_theta,0.001,1000000)
theta_grad

array([[-0.12905073],
       [ 0.10194246],
       [-0.33660882]])

In [305]:
cost_history[-1]

14045.738223882996

In [279]:
initial_theta_ba = np.random.rand(3,1)
initial_theta_ba

array([[0.42203653],
       [0.71131459],
       [0.53375735]])

In [311]:
theta_batch, c_history = minibatch_gradient_descent(X_train.values,y_theta,initial_theta_ba,0.001,10000,10)
theta_batch

array([[0.39831732],
       [0.45004059],
       [0.36964516]])

In [312]:
c_history[-1]

451.82290153355973

### Predict com o dataset de teste para o método do gradiente descendente

In [306]:
x_predict = np.c_[np.ones((X_test.shape[0],1)),X_test.values]

In [307]:
y_pred = predict(theta_grad, x_predict)

In [308]:
confusion_matrix_grad = confusion_matrix(y_test, y_pred)
print("Matriz de Confusão para o metodo do gradiente descendente:\n", confusion_matrix_grad)

Matriz de Confusão para o metodo do gradiente descendente:
 [[9 0]
 [9 0]]


In [309]:
precision_grad,recall_grad,fscore_grad,support_grad = precision_recall_fscore_support(y_test,y_pred)
print("Precisao para o metodo do gradiente descendente: ", precision_grad)
print("Recall para o metodo do gradiente descendente: ", recall_grad)
print("F-Score para o metodo do gradiente descendente: ", fscore_grad)
print("Support para o metodo do gradiente descendente: ", support_grad)

Precisao para o metodo do gradiente descendente:  [0.5 0. ]
Recall para o metodo do gradiente descendente:  [1. 0.]
F-Score para o metodo do gradiente descendente:  [0.66666667 0.        ]
Support para o metodo do gradiente descendente:  [9 9]


/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [310]:
acc_grad = accuracy_score(y_test, y_pred)
print("Acuracia para o metodo do gradiente descendente: ", acc_grad)

Acuracia para o metodo do gradiente descendente:  0.5


### Predict com o dataset de teste para o método do batch

In [313]:
x_predict = np.c_[np.ones((X_test.shape[0],1)),X_test.values]

In [314]:
y_pred = predict(theta_batch, x_predict)

In [315]:
confusion_matrix_batch = confusion_matrix(y_test, y_pred)
print("Matriz de Confusão para o metodo do gradiente batch:\n", confusion_matrix_batch)

Matriz de Confusão para o metodo do gradiente batch:
 [[0 9]
 [0 9]]


In [316]:
precision_batch,recall_batch,fscore_batch,support_batch = precision_recall_fscore_support(y_test,y_pred)
print("Precisao para o metodo do gradiente batch: ", precision_batch)
print("Recall para o metodo do gradiente batch: ", recall_batch)
print("F-Score para o metodo do gradiente batch: ", fscore_batch)
print("Support para o metodo do gradiente batch: ", support_batch)

Precisao para o metodo do gradiente batch:  [0.  0.5]
Recall para o metodo do gradiente batch:  [0. 1.]
F-Score para o metodo do gradiente batch:  [0.         0.66666667]
Support para o metodo do gradiente batch:  [9 9]


/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [317]:
acc_batch = accuracy_score(y_test, y_pred)
print("Acuracia para o metodo do gradiente batch: ", acc_batch)

Acuracia para o metodo do gradiente batch:  0.5
